In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
from benedict import benedict

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
# from benedict import benedict # https://github.com/fabiocaccamo/python-benedict#usage

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveWrapper import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# Animal `gor01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='one') # IdentifyingContext<('kdiba', 'gor01', 'one')>
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name) # 'gor01', 'one'
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['PhoHelpers', 'Spike3D-Minimal-Test', 'Unused'])

local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='two')
local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# ## Animal `vvp01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='one')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='two')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# ### Animal `pin01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='pin01', exper_name='one')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name) # no exper_name ('one' or 'two') folders for this animal.
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['redundant','showclus','sleep','tmaze'])

## Build session contexts list:
local_session_contexts_list = [local_session_parent_context.adding_context(collision_prefix='sess', session_name=a_name) for a_name in local_session_names_list] # [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>, ..., IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>]

## Initialize `session_batch_status` with the NOT_STARTED status if it doesn't already have a different status
for curr_session_basedir in local_session_paths_list:
    curr_session_status = session_batch_status.get(curr_session_basedir, None)
    if curr_session_status is None:
        session_batch_status[curr_session_basedir] = SessionBatchProgress.NOT_STARTED # set to not started if not present
        # session_batch_status[curr_session_basedir] = SessionBatchProgress.COMPLETED # set to not started if not present

session_batch_status

local_session_names_list: ['2006-6-07_16-40-19', '2006-6-08_15-46-47', '2006-6-08_21-16-25', '2006-6-09_22-24-40', '2006-6-12_16-53-46', '2006-6-13_15-22-3']


{WindowsPath('W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/two/2006-6-08_15-46-47'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/two/2006-6-13_15-22-3'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>}

In [3]:
%pdb off
# %%viztracer
basedir = local_session_paths_list[0] # NOT 3
print(f'basedir: {str(basedir)}')

# ## Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# Force write:
saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
force_reload = True

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# epoch_name_whitelist = ['maze']
epoch_name_whitelist = None
active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        # '_perform_pf_find_ratemap_peaks_computation',
                                        '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_whitelist=epoch_name_whitelist,
                                          computation_functions_name_whitelist=active_computation_functions_name_whitelist,
                                          saving_mode=saving_mode, force_reload=force_reload,
                                          skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True)

Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19
Skipping loading from pickled file because force_reload == True.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.epochs_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.position_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.spikes.mat... 

C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\session\Formats\SessionSpecifications.py:140: UserWarning: WARNING: Optional File: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.position.npy... 2006-6-07_16-40-19.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.interpolated_spike_positions.npy... 2006-6-07_16-40-19.interpolated_spike_positions.npy saved
done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.laps_info.mat... done.
setting laps object.
session.laps loaded successfully!
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.replay_info.mat... done.
session.replays loaded success

In [4]:
from neuropy.analyses.placefields import PfND
from neuropy.analyses.laps import estimate_session_laps

## 2023-04-07 - Builds the laps using estimation_session_laps(...) if needed for each epoch, and then sets the decoder's .epochs property to the laps object so the occupancy is correct.
def constrain_to_laps(curr_active_pipeline):
	""" 2023-04-07 - Constrains the placefields to just the laps, computing the laps if needed.
	Other laps-related things?
		# ??? pos_df = sess.compute_position_laps() # ensures the laps are computed if they need to be:
		# DataSession.compute_position_laps(self)
		# DataSession.compute_laps_position_df(position_df, laps_df)
	"""
	long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
	long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
	long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]

	for a_name, a_sess, a_result in zip((long_epoch_name, short_epoch_name, global_epoch_name), (long_session, short_session, global_session), (long_results, short_results, global_results)):
		a_sess = estimate_session_laps(a_sess, should_plot_laps_2d=False)
		curr_laps_obj = a_sess.laps.as_epoch_obj() # set this to the laps object
		curr_laps_obj = curr_laps_obj.get_non_overlapping()
		curr_laps_obj = curr_laps_obj.filtered_by_duration(1.0, 10.0) # the lap must be at least 1 second long and at most 10 seconds long
		# curr_laps_obj = a_sess.estimate_laps().as_epoch_obj()
		curr_active_pipeline.active_configs[a_name].computation_config.pf_params.computation_epochs = curr_laps_obj
		curr_pf1D, curr_pf2D = a_result.pf1D, a_result.pf2D

		lap_filtered_curr_pf1D = deepcopy(curr_pf1D)
		lap_filtered_curr_pf1D = PfND(spikes_df=lap_filtered_curr_pf1D.spikes_df, position=lap_filtered_curr_pf1D.position, epochs=deepcopy(curr_laps_obj), config=lap_filtered_curr_pf1D.config, compute_on_init=True)
		lap_filtered_curr_pf2D = deepcopy(curr_pf2D)
		lap_filtered_curr_pf2D = PfND(spikes_df=lap_filtered_curr_pf2D.spikes_df, position=lap_filtered_curr_pf2D.position, epochs=deepcopy(curr_laps_obj), config=lap_filtered_curr_pf2D.config, compute_on_init=True)
		a_result.pf1D = lap_filtered_curr_pf1D
		a_result.pf2D = lap_filtered_curr_pf2D
		return curr_active_pipeline

curr_active_pipeline = constrain_to_laps(curr_active_pipeline)

sane_midpoint_x: 118.93433364528494, hardcoded_track_midpoint_x: 150.0, track_min_max_x: (18.67140495721134, 256.5400722477812)
desc_crossings_x: (20,), asc_crossings_x: (20,)


In [5]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_one_step_decoder_1D, short_one_step_decoder_1D  = [results_data.get('pf1D_Decoder', None) for results_data in (long_results, short_results)]
recalculate_anyway = True

In [6]:
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_position_decoding_computation'], computation_kwargs_list=[dict(ndim=1)], enabled_filter_names=[long_epoch_name, short_epoch_name], fail_on_exception=True, debug_print=True)
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
# long_one_step_decoder_1D, short_one_step_decoder_1D  = [results_data.get('pf1D_Decoder', None) for results_data in (long_results, short_results)]
long_one_step_decoder_1D, short_one_step_decoder_1D  = [deepcopy(results_data.get('pf1D_Decoder', None)) for results_data in (long_results, short_results)]
# ds and Decoders conform between the long and the short epochs:
short_one_step_decoder_1D, did_recompute = short_one_step_decoder_1D.conform_to_position_bins(long_one_step_decoder_1D, force_recompute=True)

# ## Build or get the two-step decoders for both the long and short:
# long_two_step_decoder_1D, short_two_step_decoder_1D  = [results_data.get('pf1D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
# if recalculate_anyway or did_recompute or (long_two_step_decoder_1D is None) or (short_two_step_decoder_1D is None):
#     curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_two_step_position_decoding_computation'], computation_kwargs_list=[dict(ndim=1)], enabled_filter_names=[long_epoch_name, short_epoch_name], fail_on_exception=True, debug_print=True)
#     long_two_step_decoder_1D, short_two_step_decoder_1D  = [results_data.get('pf1D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
#     assert (long_two_step_decoder_1D is not None and short_two_step_decoder_1D is not None)

decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333
# decoding_time_bin_size = 0.03 # 0.03333333333333333
print(f'decoding_time_bin_size: {decoding_time_bin_size}')

Performing run_specific_computations_single_context on filtered_session with filter named "maze1"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
	 spikes_df[time_variable_name]: (13754,) should be less than time_window_edges: (35786,)!
	 spikes_df[time_variable_name]: (13754,) should be less than time_window_edges: (35786,)!
_execute_computation_functions(...): 
	accumulated_errors: None
Performing run_specific_computations_single_context on filtered_session with filter named "maze2"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
	 spikes_df[time_variable_name]: (18163,) should be less than time_window_edges: (40455,)!
	 spikes_df[time_variable_name]: (18163,) should be less than time_window_edges: (40455,)!
_execute_computation_functions(...): 
	accumulated_errors: None
self will be re-binned to match target_pf...
done.
self will be re-binned to match target_one_step_decoder...
	 spikes_df[time

#### Get 2D Decoders for validation and comparisons:

In [7]:
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_position_decoding_computation'], computation_kwargs_list=[dict(ndim=2)], enabled_filter_names=[long_epoch_name, short_epoch_name], fail_on_exception=True, debug_print=True)
# Make the 2D Placefields and Decoders conform between the long and the short epochs:
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
long_one_step_decoder_2D, short_one_step_decoder_2D  = [results_data.get('pf2D_Decoder', None) for results_data in (long_results, short_results)]
short_one_step_decoder_2D, did_recompute = short_one_step_decoder_2D.conform_to_position_bins(long_one_step_decoder_2D)

# ## Build or get the two-step decoders for both the long and short:
# long_two_step_decoder_2D, short_two_step_decoder_2D  = [results_data.get('pf2D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
# if recalculate_anyway or did_recompute or (long_two_step_decoder_2D is None) or (short_two_step_decoder_2D is None):
#     curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_two_step_position_decoding_computation'], computation_kwargs_list=[dict(ndim=2)], enabled_filter_names=[long_epoch_name, short_epoch_name], fail_on_exception=True, debug_print=True)
#     long_two_step_decoder_2D, short_two_step_decoder_2D  = [results_data.get('pf2D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
#     assert (long_two_step_decoder_2D is not None and short_two_step_decoder_2D is not None)
# Sums are similar:
print(f'{np.sum(long_one_step_decoder_2D.marginal.x.p_x_given_n) =},\t {np.sum(long_one_step_decoder_1D.p_x_given_n) = }') # 31181.999999999996 vs 31181.99999999999

## Validate:
assert long_one_step_decoder_2D.marginal.x.p_x_given_n.shape == long_one_step_decoder_1D.p_x_given_n.shape, f"Must equal but: {long_one_step_decoder_2D.marginal.x.p_x_given_n.shape =} and {long_one_step_decoder_1D.p_x_given_n.shape =}"
assert long_one_step_decoder_2D.marginal.x.most_likely_positions_1D.shape == long_one_step_decoder_1D.most_likely_positions.shape, f"Must equal but: {long_one_step_decoder_2D.marginal.x.most_likely_positions_1D.shape =} and {long_one_step_decoder_1D.most_likely_positions.shape =}"

## validate values:
# assert np.allclose(long_one_step_decoder_2D.marginal.x.p_x_given_n, long_one_step_decoder_1D.p_x_given_n), f"1D Decoder should have an x-posterior equal to its own posterior"
# assert np.allclose(curr_epoch_result['marginal_x']['most_likely_positions_1D'], curr_epoch_result['most_likely_positions']), f"1D Decoder should have an x-posterior with most_likely_positions_1D equal to its own most_likely_positions"

Performing run_specific_computations_single_context on filtered_session with filter named "maze1"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
	 spikes_df[time_variable_name]: (13754,) should be less than time_window_edges: (35786,)!
	 spikes_df[time_variable_name]: (13754,) should be less than time_window_edges: (35786,)!
_execute_computation_functions(...): 
	accumulated_errors: None
Performing run_specific_computations_single_context on filtered_session with filter named "maze2"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
	 spikes_df[time_variable_name]: (18163,) should be less than time_window_edges: (40455,)!
	 spikes_df[time_variable_name]: (18163,) should be less than time_window_edges: (40455,)!
_execute_computation_functions(...): 
	accumulated_errors: None
self will be re-binned to match target_pf...
done.
self will be re-binned to match target_one_step_decoder...
	 spikes_df[time

In [8]:
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent

curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_time_dependent_placefield_computation'], enabled_filter_names=[long_epoch_name, short_epoch_name, global_epoch_name], fail_on_exception=True, debug_print=True)

# long_results.pf1D_dt.reset()
# long_results.pf1D_dt._included_thresh_neurons_indx


Performing run_specific_computations_single_context on filtered_session with filter named "maze1"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
Recomputing active_epoch_time_dependent_placefields... 	 done.
Recomputing active_epoch_time_dependent_placefields2D... 	 done.
_execute_computation_functions(...): 
	accumulated_errors: None
Performing run_specific_computations_single_context on filtered_session with filter named "maze2"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
Recomputing active_epoch_time_dependent_placefields... 

ValueError: zero-size array to reduction operation fmin which has no identity

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE)

# 2023-04-11 - Review of Year's Progress


In [9]:
%matplotlib qt
import matplotlib.pyplot as plt

In [ ]:
# test for interactive
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
plotter = curr_active_pipeline.plot


In [ ]:
%matplotlib inline


In [ ]:
plotter._display_normal(active_session_configuration_context=long_epoch_name)

In [ ]:
curr_active_pipeline.plot._display_short_long_firing_rate_index_comparison()

In [ ]:
curr_active_pipeline.plot.display_short_long_firing_rate_index_comparison

In [9]:
# extended_computations_include_whitelist=None # do all
extended_computations_include_whitelist=['jonathan_firing_rate_analysis', 'short_long_pf_overlap_analyses', 'long_short_fr_indicies_analyses'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_whitelist=extended_computations_include_whitelist, include_global_functions=True, fail_on_exception=True, progress_print=True, debug_print=True)


included whitelist is specified: ['jonathan_firing_rate_analysis', 'short_long_pf_overlap_analyses', 'long_short_fr_indicies_analyses'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis missing.
	 encountered error: 'jonathan_firing_rate_analysis'
Traceback (most recent call last):
  File "C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Batch\NonInteractiveWrapper.py", line 423, in batch_extended_computations
    curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
  File "C:\Users\pho\repos\Spike3DWorkEnv\pyPhoCoreHelpers\src\pyphocorehelpers\DataStructure\dynamic_parameters.py", line 34, in __getitem__
    return self._mapping[key]
KeyError: 'jonathan_firing_rate_analysis'

.
	 Recomputing jonathan_firing_rate_analysis...
Time window 13 has no spikes.
Time window 

In [11]:
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_short_long_firing_rate_analyses'], fail_on_exception=True, debug_print=False) # fail_on_exception MUST be True or error handling is all messed up 
print(f'\t done.')
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']

	 done.


In [25]:
y_frs_index

{2: 0.2476010628496219,
 3: 0.7716618563899214,
 4: -0.5865695778495489,
 5: 0.6115488327962585,
 6: -0.036315446265052904,
 7: 0.09372605049726528,
 8: -0.1535207898857728,
 9: 0.06398167407285156,
 10: 0.451505512674487,
 11: 0.1872898525252567,
 12: -0.15672894431893034,
 13: 0.30323557677214763,
 14: 0.10915621672599213,
 15: 0.4441317561544812,
 16: -0.8352173806901315,
 17: -0.14843880261574782,
 18: 0.39256752841687714,
 19: 0.5027728411545584,
 20: -0.30741169848784045,
 21: 0.004844883598976626,
 22: -0.07816952934647496,
 23: 0.21976004411374606,
 24: -0.43046950016528807,
 25: 0.18091944837293042,
 26: 0.5961964967369102,
 27: 0.3951136421630797,
 28: -0.17086513554512095,
 29: 0.7256424869222341,
 30: 0.016872618135481906,
 31: -0.3777095810762753,
 32: -0.021189848363497167,
 33: -0.6371416551897979,
 34: 0.255603138389498,
 35: -0.963382022583852,
 36: -0.06087253384175859,
 37: 0.008395699233839198,
 38: 0.39161120210371114,
 39: 0.07445514246837664,
 40: 0.2305138800752

In [12]:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis['neuron_replay_stats_df'], curr_jonathan_firing_rate_analysis['rdf']['rdf'], curr_jonathan_firing_rate_analysis['rdf']['aclu_to_idx'], curr_jonathan_firing_rate_analysis['irdf']['irdf']

In [13]:
short_long_pf_overlap_analyses = curr_active_pipeline.global_computation_results.computed_data.short_long_pf_overlap_analyses
conv_overlap_dict = short_long_pf_overlap_analyses['conv_overlap_dict']
conv_overlap_scalars_df = short_long_pf_overlap_analyses['conv_overlap_scalars_df']
prod_overlap_dict = short_long_pf_overlap_analyses['product_overlap_dict']
relative_entropy_overlap_dict = short_long_pf_overlap_analyses['relative_entropy_overlap_dict']
relative_entropy_overlap_scalars_df = short_long_pf_overlap_analyses['relative_entropy_overlap_scalars_df']

In [26]:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
long_short_fr_indicies_analysis_results

DynamicParameters({'long_laps': 35 epochs
array([[   6.16015557,    9.86222087],
       [  44.90800901,   50.90700754],
       [  65.77649368,   72.21441935],
       [  85.04824574,   91.07698121],
       [ 107.60043131,  112.98791783],
       [ 243.90116152,  250.66060254],
       [ 308.59855432,  313.92546103],
       [ 357.89826156,  365.10301947],
       [ 370.32891887,  376.02766027],
       [ 399.69181712,  406.82152611],
       [ 414.95878914,  422.10840468],
       [ 428.32007766,  435.32110064],
       [ 441.20176578,  445.84284885],
       [ 465.23769588,  470.1561804 ],
       [ 481.14877381,  489.66920721],
       [ 491.29490878,  497.08095637],
       [ 502.15970705,  509.78201099],
       [ 513.9409439 ,  520.1870847 ],
       [ 528.38670931,  533.08637538],
       [ 536.05942474,  541.86049441],
       [ 546.23861441,  551.23128769],
       [ 555.49380839,  563.48416846],
       [ 584.35461921,  592.32771465],
       [ 644.21484328,  652.13313427],
       [ 684.4117891 ,

In [12]:
%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

graphics_output_dict = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', active_identifying_session_ctx)
fig, axs, plot_data = graphics_output_dict['fig'], graphics_output_dict['axs'], graphics_output_dict['plot_data']
neuron_df, rdf, aclu_to_idx, irdf = plot_data['df'], plot_data['rdf'], plot_data['aclu_to_idx'], plot_data['irdf']
# Grab the output axes:
curr_axs_dict = axs[0]
curr_firing_rate_ax, curr_lap_spikes_ax, curr_placefield_ax = curr_axs_dict['firing_rate'], curr_axs_dict['lap_spikes'], curr_axs_dict['placefield'] # Extract variables from the `curr_axs_dict` dictionary to the local workspace
## TODO 2023-04-11 - Set Window Title for '_display_batch_pho_jonathan_replay_firing_rate_comparison'

include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


In [19]:
# Plot long|short firing rate index:
fig_save_parent_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Pho-Kamran-Meetings\Results from 2023-04-11')
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', curr_active_pipeline.sess.get_context(), fig_save_parent_path=fig_save_parent_path)
# plt.close() # closes the current figure

Saving plot_long_short_firing_rate_indicies results results : E:\Dropbox (Personal)\Active\Kamran Diba Lab\Pho-Kamran-Meetings\Results from 2023-04-11\kdiba_gor01_one_2006-6-09_1-22-43_long_short_firing_rate_indicies.png... done.


MatplotlibRenderPlots({'name': 'display_short_long_firing_rate_index_comparison', 'figures': <Figure size 850x725 with 1 Axes>, 'axes': (<Axes: title={'center': 'Computed long ($L$)|short($S$) firing rate indicies'}, xlabel='$\\frac{L_{R}-S_{R}}{L_{R} + S_{R}}$', ylabel='$\\frac{L_{\\theta}-S_{\\theta}}{L_{\\theta} + S_{\\theta}}$'>,), 'plot_data': {}})

In [ ]:
# 2023-04-11 - Debug why `_display_short_long_firing_rate_index_comparison` is empty
# Plot long|short firing rate index:

long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
long_short_fr_indicies_analysis_results


In [27]:
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', curr_active_pipeline.sess.get_context(), fig_save_parent_path=fig_save_parent_path)

Saving plot_long_short_firing_rate_indicies results results : E:\Dropbox (Personal)\Active\Kamran Diba Lab\Pho-Kamran-Meetings\Results from 2023-04-11\kdiba_gor01_one_2006-6-09_1-22-43_long_short_firing_rate_indicies.png... done.


MatplotlibRenderPlots({'name': 'display_short_long_firing_rate_index_comparison', 'figures': <Figure size 850x725 with 1 Axes>, 'axes': (<Axes: title={'center': 'Computed long ($L$)|short($S$) firing rate indicies'}, xlabel='$\\frac{L_{R}-S_{R}}{L_{R} + S_{R}}$', ylabel='$\\frac{L_{\\theta}-S_{\\theta}}{L_{\\theta} + S_{\\theta}}$'>,), 'plot_data': {}})

In [ ]:
active_identifying_session_ctx, active_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)


In [ ]:
batch_extended_programmatic_figures(curr_active_pipeline)